# Reubens Work to Oracle

In [11]:
import sys
import os
import warnings
warnings.filterwarnings("ignore")
from collections import OrderedDict
import numpy as np
import pandas as pd

In [12]:
def loadHeader(log_path, print_missing_requireds = True):
    # Load the column name qualifiers
    header = pd.read_excel(log_path, sheet_name = 'Metadata', usecols= 'A:C')

    if print_missing_requireds:
        # Check all required fields are filled out
        for _, row in header.iterrows():
            if pd.isna(row['Value']) and row['Value Required'] == True:
                print('Compulsory field missing: populate ' + row['Attribute'] + ' for hole ' + str(header[header['Attribute'] == 'Hole Registration number']['Value'].values[0]) + ' before continuing')
    return header

In [13]:
def prepareDigitalLog(log_path):
    def createColumnNames(log_path):
        # Load the column name qualifiers/prefixes
        col_prefixes = pd.read_excel(log_path, sheet_name = 'Detailed Logging Form', skiprows = 0).columns

        # load the log columns
        log_cols = pd.read_excel(log_path, sheet_name = 'Detailed Logging Form', skiprows = 1).columns

        # combine the qualifier and colunm names to make better column names
        colname_lu = pd.Series(col_prefixes).apply(lambda x: x if x[0:8] != 'Unnamed:' else np.nan).fillna(method = 'ffill')
        new_cols = [colname_lu[i] + ': ' + col.strip() for i, col in enumerate(log_cols)]

        return new_cols
    
    # load the log data
    log = pd.read_excel(log_path, sheet_name = 'Detailed Logging Form', skiprows = 1)
    log.columns = createColumnNames(log_path)
    # remove the error in the template where a blank cell is populated with "  ferromagnesian"
    if log.loc[1, 'assemblage: maghaemite'] == '  ferromagnesian':
        log.loc[1, 'assemblage: maghaemite'] = np.nan    
    # if any of top, bottom and em1 lithology are null, drop
    log = log[log[['position: From (m bgl)','position: To (m bgl)', 'Extra Fields for Oracle: EM1 Lithology Name (eg, soil, muddy sand, sandstone), see lookup tab']].notna().all(axis = 1)]# 

    return log

In [14]:
def getENO(header):
    return header[header['Attribute'] == 'ENO']['Value'].values[0]

def getBoreholeName(header):
    return header[header['Attribute'] == 'Hole Registration number']['Value'].values[0]

def getCollectionName(header):
    return header[header['Attribute'] == 'Collection Name']['Value'].values[0]

def getSourceDocument(header):
    return header[header['Attribute'] == 'File Location']['Value'].values[0]

def getAccessGroup(header):
    return header[header['Attribute'] == 'Access Group']['Value'].values[0]

def getAccessCode(header):
    return header[header['Attribute'] == 'Access Code']['Value'].values[0]

def getConfidentialUntilDate(header):
    return header[header['Attribute'] == 'Confidential Until Date']['Value'].values[0]

def getQAStatus(header):
    return header[header['Attribute'] == 'QA Status']['Value'].values[0]

def getActivityCode(header):
    return header[header['Attribute'] == 'Activity Code']['Value'].values[0]

def getSampleID(header, row_num):
    return str(getBoreholeName(header)) + '.lithostrat.' + str(row_num + 1)

def getTemplateVersion(header):
    if 'Template Version' not in header.index:
        return 'old'
    else:
        return header['Template Version']

def getLithologyName(interval, column_name):
    lith = interval[column_name]
    if lith == '?':
        lith = 'unknown'
    elif pd.isna(lith):
        pass
    else:
        lith = lith.strip().lower()
    return lith
    

def useLithologyMaterialLU(em_lithology_name, column):
    if "material" in column.lower():
        column = 'MATERIAL CLASS'
    elif "group" in column.lower():
        column = 'LITHOLOGY GROUP'
    lu_path = r'\\prod.lan\active\proj\futurex\StuartCorridor\ScriptsTools\Consolidated_BH_Data\lithtype_lithgroup_materialclass_LU.xlsx'
    lu = pd.read_excel(lu_path, sheet_name='Mappings')
    lu = lu[lu.notna().all(axis = 1)]
    try:
        return lu[lu['LITHOLOGY TYPE'] == em_lithology_name][column].values[0]
    except:
        raise ValueError('{} is not filled out for {} in the lookup mapping file.\nPlease update the mapping file found at {}'.format(column, em_lithology_name, lu_path))
        return None

def makeBOREHOLEINTERVALCOLLECTION(header, template_path):
    s = 'Creating borehole header...'
    print(s, end = '', flush = True)
    
    # define each of the required fields for the bulk loader
    eno = getENO(header)
    borehole_name = getBoreholeName(header)
    depth_reference_point_id = header[header['Attribute'] == 'Depth Reference Point Id']['Value'].values[0]
    interval_collection_id =   ''  # will always be blank only required for existing holes
    collection_name = getCollectionName(header)
    collection_type = header[header['Attribute'] == 'Collection Type']['Value'].values[0]
    originator = 1328 # Mike Barnes' origno
    preferred_collection = header[header['Attribute'] == 'Preferred Collection']['Value'].values[0]
    collection_source_document = getSourceDocument(header)
    source_comments = header[header['Attribute'] == 'Source Comments']['Value'].values[0]

    # Put all the fields together in a list (in the correct order)
    bore_int_coll_entry = [eno, borehole_name, depth_reference_point_id, interval_collection_id, collection_name, collection_type, originator, preferred_collection, collection_source_document, source_comments]
    # Create a new entry at the bottom of the list with the list as data
    df_dict['BOREHOLE INTERVAL COLLECTION'].loc[len(df_dict['BOREHOLE INTERVAL COLLECTION'])] = bore_int_coll_entry
    print('Done')

In [15]:
def makeDOWNHOLEINTERVALSANDSAMPLES(header, row_num, interval):
    def makeComments(interval):
        if not pd.isna(interval['lithology: eg. Sand, fine; Clay; interbedded sand and silt and clay etc.']):
            c1 = interval['lithology: eg. Sand, fine; Clay; interbedded sand and silt and clay etc.']
        else:
            c1 = None
        if not pd.isna(interval['assemblage: Other Constituents (eg other minerals, organics, fossils etc)']):
            c2 = interval['assemblage: Other Constituents (eg other minerals, organics, fossils etc)']
        else:
            c2 = None
        if not pd.isna(interval['comments: Any additional observations']):
            c3 = interval['comments: Any additional observations']
        else:
            c3 = None

        comments = []
        for comment in [c1,c2,c3]:
            if comment is not None:
                if comments == []:
                    comments = comment
                else:
                    comments += ('. ' + comment)
        return comments  
    
    # Define the required fields
    eno = getENO(header)
    collection_no =  '' # not needed as are creating new rows in Oracle
    collection_name = getCollectionName(header)
    interval_id = '' # not needed as are creating new rows in Oracle
    intervalno = '' # not needed as are creating new rows in Oracle
    top = interval['position: From (m bgl)']
    bottom = interval['position: To (m bgl)']
    sampleno = '' # Oracle assigned primary key, always blank for new entries
    sampleid = getSampleID(header, row_num)
    logging_date = '3000-01-01' # header[header['Attribute'] == 'Logging Date']['Value'].values[0]
    parent_sampleno = '' # blank because these are new entries and the samples thus won't have a parent sample
    parent_sample_id = '' # as above
    access_group = getAccessGroup(header)
    access_code = getAccessCode(header)
    confidential_until_date = getConfidentialUntilDate(header)
    qa_status = getQAStatus(header)
    activity_code = getActivityCode(header)
    sample_type = 'borehole specimen' # header[header['Attribute'] == 'Sample Type']['Value'].values[0]
    sampling_method = 'drilling' # header[header['Attribute'] == 'Collection Name']['Value'].values[0]
    material_class = useLithologyMaterialLU(getLithologyName(interval, 'Extra Fields for Oracle: EM1 Lithology Name (eg, soil, muddy sand, sandstone), see lookup tab'), 'MATERIAL CLASS')
    sample_preparation = header[header['Attribute'] == 'Sample Preparation']['Value'].values[0]
    project = header[header['Attribute'] == 'Project']['Value'].values[0]
    specimen_storage_location = header[header['Attribute'] == 'Core Storage Location']['Value'].values[0]
    storage_date = header[header['Attribute'] == 'Storage Date']['Value'].values[0]
    comments = makeComments(interval)
    igsn = header[header['Attribute'] == 'IGSN']['Value'].values[0]
    specimen_mass = header[header['Attribute'] == 'Specimen Mass']['Value'].values[0]
    mass_uom = header[header['Attribute'] == 'Mass UOM']['Value'].values[0]
    source = getSourceDocument(header)

    # Add the fields, in order, in to a list
    downhole_int_samps_entry = [eno, collection_no, collection_name, interval_id, intervalno, top, bottom, sampleno, sampleid, logging_date, parent_sampleno, parent_sample_id, access_group, access_code, confidential_until_date, qa_status, activity_code, sample_type, sampling_method, material_class, sample_preparation, project,
                         specimen_storage_location, storage_date, comments, igsn, specimen_mass, mass_uom, source]
    # Add the fields as a new row in the relevant dataframe
    df_dict['DOWNHOLE INTERVALS AND SAMPLES'].loc[len(df_dict['DOWNHOLE INTERVALS AND SAMPLES'])] = downhole_int_samps_entry

In [16]:
def makeEARTHMATERIAL(interval, header, row_num):
    def makeAssemblageDescription(interval):
        # Proportion term lookup, transforming from the abbreviation on the logging sheet to the Oracle lookup constrained value
        assemblage_cols = [col for col in log.columns if col[0:10] == 'assemblage'][0:-3]
        s = ''
        for mineral, prop_term in interval[assemblage_cols].sort_values().iteritems():
            if not pd.isna(prop_term):
                if s == '':
                    s = proportion_term_lu[prop_term.strip()] + ' ' + mineral.split(':')[1].strip()
                else:
                    s += ', ' + proportion_term_lu[prop_term.strip()] + ' ' + mineral.split(':')[1].strip()
        if not pd.isna(interval['assemblage: Other Constituents (eg other minerals, organics, fossils etc)']):
            s += '. ' + interval['assemblage: Other Constituents (eg other minerals, organics, fossils etc)']
        return s
    
    def makeMineralQualifiers(interval):
        qualifier_lu = {'lithic fragments':'lithic'}
        qualifer_cols = [col for col in interval.index if col[0:10] == 'assemblage' and col.split(':')[1].strip() in qualifier_lu.keys()]
        qualifiers = interval[qualifer_cols].sort_values()
        qualifier_values = []
        for i, (mineral, prop_term) in enumerate(qualifiers.iteritems()):
            if not pd.isna(prop_term) and i < 3:
                mineral = mineral.split(':')[1].strip()
                qualifier_values.append(qualifier_lu[mineral])
        qualifier_values = qualifier_values + [''] * (3 - len(qualifier_values))
        return qualifier_values
    
    proportion_term_lu = {'D': 'dominant', 'M' : 'minor', 'T' : 'trace'}
    
    # #### EARTH MATERIALS  ####
    collection_name  = getCollectionName(header)
    sampleno = '' # not needed as it will be automatically generated from sampleid
    sampleid = getSampleID(header, row_num)
    access_code = getAccessCode(header)
    confidential_until_date = getConfidentialUntilDate(header)
    qa_status = getQAStatus(header)
    activity_code  = getActivityCode(header)
#     lithology_group defined in loop
#     lithology_name defined in loop
    qualifier_1, qualifier_2, qualifier_3 = makeMineralQualifiers(interval)
    material_description  = makeAssemblageDescription(interval)   # concatenation of assemblage descriptors
#     mode_of_occurence defined in loop
    proportion_term = ''  # numbers used instead
    # lower_proportion_percent defined in loop
    # upper_proportion_percent defined in loop
    if getTemplateVersion(header) != 'old':
        em_confidence = interval['Confidence: Lithology Type Confidence']
    else:
        em_confidence = ''
    # Extract the earth material details from the interval. There could be two earth materials
    em_fields = [col for col in interval.index if col.split(':')[1].strip()[0:2] == 'EM']
    # build a dict with the keys of the em number, with the value of a dict for lith name and lith proportion
    EMs = {emname : {'lith_type': None,'proportion': None} for emname in {em_field.split(':')[1].strip()[0:3] for em_field in em_fields}}
    # loop through the EM fields, and harvest the data and store it in the dict
    for em_field in em_fields:
        column_name = list(map(str.strip, em_field.split(':')[1].split(' '))) # EM1 or EM2
        if column_name[3] == 'Name':
            EMs[column_name[1]]['lith_type'] = getLithologyName(interval, em_field)
        if column_name[3] == '(%)':
            EMs[column_name[1]]['proportion'] = interval[em_field]

    for key, value in EMs.items():
        lithology_name = value['lith_type']
        if pd.isna(lithology_name):
            continue
        lithology_group = useLithologyMaterialLU(lithology_name, 'group')
        lower_proportion_percent = upper_proportion_percent = value['proportion']
        if lithology_group == 'organic':
            mode_of_occurence = ''
        else:
            mode_of_occurence = 'bed'
        
        earth_materials_entry = [collection_name, sampleno, sampleid, access_code, confidential_until_date, qa_status, activity_code, '', lithology_group, lithology_name,
                                 qualifier_1, qualifier_2, qualifier_3, material_description, mode_of_occurence, proportion_term, lower_proportion_percent,
                                 upper_proportion_percent, em_confidence]
        df_dict['EARTH MATERIALS'].loc[len(df_dict['EARTH MATERIALS'])] = earth_materials_entry

In [17]:
def makeEXTRAPROPERTIES(interval, header, row_num):
    def makeExtraProperty(geo_property, text_value = '', numerical_value = '', uom = '', proportion_term = '', prop_confidence = '', comment = ''):
        """
        Function for adding an extra property of an earth material to the relevant table.
        """
        extra_properties_entry = [sampleid, sampleno, datano, geo_property, text_value, numerical_value, uom, proportion_term, prop_confidence,
                                  comment, access_code, confidential_until_date, qa_status, activity_code, access_group] # for when proportion term is available
#         extra_properties_entry = [sampleid, sampleno, datano, geo_property, text_value, numerical_value, uom, prop_confidence, comment, access_code, confidential_until_date, qa_status, activity_code, access_group]
        df_dict['EXTRA PROPERTIES'].loc[len(df_dict['EXTRA PROPERTIES'])] = extra_properties_entry
        
    # functions for more complicated conversion from the log sheet to the Oracle lookup constrained format
    def makeGrainsizeProperties(interval):
        grainsize_lu = {'clay.1' :           ['clay (<0.002 mm)',''],
                        'silt':              ['silt (0.002-0.0625 mm)',''],
                        'very fine - fine':  ['fine sand (0.125-0.5mm)','very fine and fine classes combined during logging'],
                        'medium':            ['medium sand (0.25-0.5 mm)',''],
                        'coarse-very coarse':['coarse sand (0.5-1 mm)','coarse and very coarse classes combined during logging'],
                        'granule':           ['granule (2-4 mm)',''],
                        'pebble':            ['pebble (4-64 mm)',''],
                        'cobble':            ['cobble (64-256 mm)''']}
        grainsize_cols = [col for col in log.columns if col[0:10] == 'grain size']
        for col in grainsize_cols:
            if not pd.isna(interval[col]):
                makeExtraProperty('grainsize (Udden-Wentworth)', proportion_term = proportion_term_lu[interval[col]], text_value = grainsize_lu[col.split(':')[1].strip()][0], comment = grainsize_lu[col.split(':')[1].strip()][1], prop_confidence = prop_confidence)

    def makeRoundingProperties(interval):
        rounding_lu = {'VA':'very angular', 'A' :'angular', 'SA':'subangular', 'SR': 'subrounded', 'R':'rounded', 'WR':'well rounded'}
        rounding_heirachy = OrderedDict([('A',0), ('SA',1),('SR',2),('R',3), ('WR',4)])
        log_rounding = interval['round: Rounding']
        if pd.isna(log_rounding):
            return
        if '-' not in log_rounding:
            makeExtraProperty('rounding', proportion_term = 'all', text_value = rounding_lu[log_rounding], prop_confidence = prop_confidence)
        else:    
            round1, round2 = list(map(str.strip,log_rounding.split('-')))
            round1, round2 = rounding_heirachy[round1], rounding_heirachy[round2]
            start, end = min(round1, round2), max(round1, round2)
            for key in list(rounding_heirachy.keys())[start: end + 1]:
                makeExtraProperty('rounding', proportion_term = 'present', text_value = rounding_lu[key], prop_confidence = prop_confidence)

    def makeMineralProperties(interval):
        mineral_lu = {'quartz':'quartz',
                      'iron oxides':'iron oxide',
                      'maghaemite':'maghemite',
                      'calcite':'calcite',
                      'gypsum':'gypsum',
                      'mica' :'mica',
                      'feldspars':'feldspar',
                      'glauconite':'glauconite',
                      'sulphides':'sulphide'}
        assemblage_cols = [col for col in log.columns if col[0:10] == 'assemblage'][0:-3]
        for mineral, prop_term in interval[assemblage_cols].sort_values().iteritems():
            if not pd.isna(prop_term):
                mineral = mineral.split(':')[1].strip()
                if mineral in mineral_lu.keys():
                    makeExtraProperty('mineral', proportion_term = proportion_term_lu[prop_term], text_value = mineral_lu[mineral], prop_confidence = prop_confidence)
    
    def makeColourProperties(interval):
        # define the columns related to Munsell Colours
        colour_cols = [col for col in log.columns if col[0:7] == 'colour:'][0:3]
        
        # create flag for if colours are grading or not
        gradingColour = True if interval['Colour: Does the colour grade from Dominant to Minor (Y/N)'] == 'Y' else False
            
        # if the colours are described as grading, make the comment string    
        if gradingColour:
            s = ''
            for col in colour_cols:
                if not pd.isna(interval[col]):
                    if s == '':
                        s = 'colour grading from ' + interval[col] + ' at top'
                    else:
                        s = s + ' to ' + interval[col]
        # loop through colours and add to Extra properties, with with proportion term and no grading desciption or with grading desc but no propotion
        for i, col in enumerate(colour_cols):
            if not pd.isna(interval[col]):
                if gradingColour:
                    proportion = ''
                    comment = s
                else:
                    if i == 0:
                        proportion = 'dominant'
                    else:
                        proportion = 'minor'
                    comment = ''
                makeExtraProperty('colour', proportion_term = proportion, text_value = interval[col], comment = comment, prop_confidence = prop_confidence)
      
        
    # Proportion term lookup, transforming from the abbreviation on the logging sheet to the Oracle lookup constrained value
    proportion_term_lu = {'D': 'dominant',
                          'M' : 'minor',
                          'T' : 'trace'}
    # #### EXTRA PROPERTIES INPUTS ####
    sampleid  = getSampleID(header, row_num)
    sampleno = ''  #not required
    datano = '' # this will always be blank as is only used for updating existing entires
    # proportion_term to be defined in function call
#     geo_propertyto be defined in function call 
#     text_value to be defined in function call
#     numerical_value  to be defined in function call
#     text_value to be defined in function call
#     uom to be defined in function call
    if getTemplateVersion(header) != 'old':
        prop_confidence = interval['Confidence: Extra Properties Confidence']
    else:
        prop_confidence = ''
#     comment to be defined in function call
    access_code = getAccessCode(header)    
    confidential_until_date = getConfidentialUntilDate(header)    
    qa_status = getQAStatus(header)    
    activity_code = getActivityCode(header)    
    access_group = getAccessGroup(header)

    # colours
    makeColourProperties(interval)
    
    # patterns
    colour_pattern_lu = {'M': 'uniform',
                 'BD' : 'striped',
                 'SK' : 'striped',
                 'BL' : 'mottled',
                 'MT' : 'mottled',
                 'SP' : 'mottled'}
    if not pd.isna(interval['Colour: pattern']):
        makeExtraProperty('colour pattern', text_value = colour_pattern_lu[interval['Colour: pattern']], prop_confidence = prop_confidence)

    # minerals
    makeMineralProperties(interval)
        
    # grainsize
    makeGrainsizeProperties(interval)

    # sorting
    if not pd.isna(interval['sort: Sorting']):
        sorting_lu = {'P': 'poorly sorted',
              'M' : 'moderately sorted',
              'W' : 'well sorted'}
        makeExtraProperty('sorting', text_value = sorting_lu[interval['sort: Sorting']], prop_confidence = prop_confidence)

    # rounding
    makeRoundingProperties(interval)

    # bedding
    if not pd.isna(interval['bed: Bedding']):
        bedding_lu = {'VTL' : 'laminated (<10 mm)',
              'L' : 'laminated (<10 mm)',
              'VTB' : 'very thin (10-30 mm)',
              'TB' : 'thin (30-100 mm)',
              'MB' : 'medium (100-300 mm)',
              'FB' : 'thick (300-1000 mm)',
              'VFB' : 'very thick (>1000 mm)'}
        makeExtraProperty('bedding thickness', text_value = bedding_lu[interval['bed: Bedding']], prop_confidence = prop_confidence)

    # sedimentary structures
    if not pd.isna(interval['structures: Sedimentary Features']):
        sed_strucs = list(map(str.strip,interval['structures: Sedimentary Features'].split(',')))
        for struc in sed_strucs:
            if pd.isna(struc):
                continue
            else:
                makeExtraProperty('sedimentary features', text_value = struc)
    
    # boundary
    if not pd.isna(interval['boundary: Lower boundary type']):
        makeExtraProperty('base contact character', text_value = interval['boundary: Lower boundary type'], prop_confidence = prop_confidence)

    # weathering
    if not pd.isna(interval['wth: Weathering']):
        weathering_lu = {'R' : 'completely weathered',
                         'X' : 'very highly weathered',
                         'D' : 'moderately weathered',
                         'S' : 'slightly weathered',
                         'F' : 'fresh'}
        makeExtraProperty('weathering degree', text_value = weathering_lu[interval['wth: Weathering']], prop_confidence = prop_confidence)    
    
    # induration
    if not pd.isna(interval['ind: ferrugenous']):
        ferrugenous_lu = {'N' : ['induration style','ferruginous nodules', ''],
                          'P' : ['sedimentary features','pisolites', 'ferrugenous'],
                          'L' : ['induration style', 'ferruginous induration', ''],
                          'C' : ['induration style', 'ferruginous induration', '']}
        makeExtraProperty(ferrugenous_lu[interval['ind: ferrugenous']][0], text_value = ferrugenous_lu[interval['ind: ferrugenous']][1], comment = ferrugenous_lu[interval['ind: ferrugenous']][2], prop_confidence = prop_confidence)

    if not pd.isna(interval['ind: calcareous']):
        calcareous_lu = {'N' : ['induration style','calcareous nodules', ''],
                  'P' : ['sedimentary features','pisolites', 'calcareous'],
                  'L' : ['induration style', 'calcareous induration', ''],
                  'C' : ['induration style', 'calcareous induration', '']}
        makeExtraProperty(calcareous_lu[interval['ind: calcareous']][0], text_value = calcareous_lu[interval['ind: calcareous']][1], comment = calcareous_lu[interval['ind: calcareous']][2], prop_confidence = prop_confidence)

    if not pd.isna(interval['ind: siliceous']):
        siliceous_lu = {'N' : ['induration style','siliceous nodules', ''],
                  'P' : ['sedimentary features','pisolites', 'siliceous'],
                  'L' : ['induration style', 'siliceous induration', ''],
                  'C' : ['induration style', 'siliceous induration', '']}
        makeExtraProperty(siliceous_lu[interval['ind: siliceous']][0], text_value = siliceous_lu[interval['ind: siliceous']][1], comment = siliceous_lu[interval['ind: siliceous']][2], prop_confidence = prop_confidence)

    # moisture
    if not pd.isna(interval['wet: Moisture']):
        moisture_lu = {'W' : 'wet',
                       'M' : 'moist',
                       'D' : 'dry'}
        makeExtraProperty('moisture', text_value = moisture_lu[interval['wet: Moisture']], prop_confidence = prop_confidence)

In [18]:
def captureSTRATIGRAPHY(interval, header, row_num):
    collection_name = 'Bore Report Stratigraphy'
    sampleno = '' # Oracle assigned primary key, always blank for new entries
    sampleid = getSampleID(header, row_num)
    access_code = getAccessCode(header)    
    confidential_until_date = getConfidentialUntilDate(header)    
    qa_status = getQAStatus(header)
    activity_code = getActivityCode(header)    
    formal_unit_name = ''
    informal_unit_name = interval['Strat: Stratigraphy']
    ga_preferred = 'Y'
    informal_grouping = ''
    geological_province = ''
    min_numeric_age = ''
    min_age_name = ''
    max_numeric_age = ''
    max_age_name = ''
    age_method = ''
    top_contact_type = ''
    top_contact_character = ''
    top_contact_name = ''
    base_contact_type = ''
    base_contact_character = ''
    base_contact_name = ''
    remarks = ''
    confidence = ''
    
    # Add the fields, in order, in to a list
    strat_entry = [collection_name, sampleno, sampleid, access_code, confidential_until_date, qa_status, activity_code, '', formal_unit_name, informal_unit_name,
    ga_preferred, informal_grouping, geological_province, min_numeric_age, min_age_name, max_numeric_age, max_age_name, age_method, top_contact_type,
    top_contact_character, top_contact_name, base_contact_type, base_contact_character, base_contact_name, remarks, confidence]
    
    # Add the fields as a new row in the relevant dataframe
    df_dict['STRATIGRAPHY'].loc[len(df_dict['STRATIGRAPHY'])] = strat_entry

In [19]:
def processEntireHole(log_path):
        # load log header data
    s = 'Loading header details...'
    print(s, end = '', flush = True)
    header = loadHeader(log_path)
    hole = getBoreholeName(header)
    print('Done')
    print('Starting Oracle bulk loader processing for', log_path)

    # load and reformat log sheet transcription
    s = 'Loading log sheet...'
    print(s, end = '', flush = True)
    log = prepareDigitalLog(log_path)
    print('Done')
    
    if len(log) == 0:
        print('No valid lithology log entries for {}'.format(hole))
        print('-----------------------------------')
        return

    # make the borehole interval collection row (1 row per hole)
    makeBOREHOLEINTERVALCOLLECTION(header, template_path)
    s = 'Creating downhole intervals and samples, earth materials and extra properties...'
    print(s, end = '', flush = True)
    for row_num, interval in log.iterrows():
        # make the samples rows (1 row per described interval)
        makeDOWNHOLEINTERVALSANDSAMPLES(header, row_num, interval)

        # make the earth materials row (up to 2 per sample)
        makeEARTHMATERIAL(interval, header, row_num)

        # make the extra property rows (potentially 10 or more per earth material)
        makeEXTRAPROPERTIES(interval, header, row_num)
        
        if pd.notna(interval['Strat: Stratigraphy']):
            captureSTRATIGRAPHY(interval, header, row_num)

    print('Done')
    print(hole, 'is processed and ready for loading into Oracle')
    print('-----------------------------------')

In [20]:
# define the template location
template_path = r"\\prod.lan\active\proj\futurex\Common\Working\Mike\GitHub\mjb_work_code\DataMan_Tools\Borehole-Samples-downhole intervals-geological observations bulk_loading_template_2019.01.xlsx"

# load the destination worksheets from the bulk loader (as dataframes to store results into)
s = 'Creating destination dataframes...'
print(s, end = '', flush = True)
df_dict = {sheet : pd.read_excel(template_path, sheet_name = sheet, skiprows = 4) for sheet in ['BOREHOLE INTERVAL COLLECTION', 'DOWNHOLE INTERVALS AND SAMPLES',
                                                                                                'EARTH MATERIALS', 'STRATIGRAPHY','EXTRA PROPERTIES']}
# remove the dummy data in the bulk loader
df_dict = {key : value.drop(value.index) for key, value in df_dict.items()}

print('Done')
print('-----------------------------------')

Creating destination dataframes...Done
-----------------------------------


## Build List of spreadsheets to Import

In [21]:
first_root_dir = r'\\prod.lan\active\proj\futurex\StuartCorridor\Working\Reuben\Boreholes_stratlith\\'

processed = pd.DataFrame(columns = ['bore_name','file_path','hasLithology','hasStratigraphy'])


for cur_dir, sub_dirs, files in os.walk(first_root_dir):
    for file in files:
        if file.startswith('RN0') and file.endswith('.xlsx'):
            bore_name = file.split('_')[0]
            file_path = os.path.join(cur_dir, file)
            df = pd.read_excel(file_path, sheet_name = 'Detailed Logging Form', skiprows = 1)
            hasLithology = True if ((df['From (m bgl)'].notna().sum() >0) and (df['EM1 Lithology Name (eg, soil, muddy sand, sandstone), see lookup tab'].notna().sum()  > 0)) else False
            hasStratigraphy = True if df['Stratigraphy'].notna().sum() > 0 else False
            processed.loc[len(processed)] = [bore_name, file_path, hasLithology, hasStratigraphy]
# Drop Reubens master files that have been imported as part of this process
processed = processed[processed['bore_name'] != 'RN0']
processed = processed[processed['bore_name'].notna()].reset_index(drop = True)
print('{} holes have been checked'.format(len(processed)))
processed = processed[processed['hasLithology'] | processed['hasStratigraphy']]
print('{} holes have some useful data'.format(len(processed)))
processed['error'] = ''
processed.head()



578 holes have been checked
455 holes have some useful data


,bore_name,file_path,hasLithology,hasStratigraphy,error
0,RN029653,\\prod.lan\active\proj\futurex\StuartCorridor\...,True,False,
1,RN029656,\\prod.lan\active\proj\futurex\StuartCorridor\...,True,False,
2,RN029660,\\prod.lan\active\proj\futurex\StuartCorridor\...,True,False,
3,RN029662,\\prod.lan\active\proj\futurex\StuartCorridor\...,True,False,
4,RN029663,\\prod.lan\active\proj\futurex\StuartCorridor\...,True,False,


In [26]:
processed[processed.duplicated('bore_name', keep = False)].sort_values('bore_name')

In [22]:
for i, row in processed.iterrows():
#     if hole_name == 'RN010703':
#         processEntireHole(path) 
#     if i < 100:
#     if row['bore_name'] == 'RN015136':
    try:
        processEntireHole(row['file_path'])
        processed.loc[i, 'error'] = 'No'
    except Exception as e:
        processed.loc[i, 'error'] = str(e)
#     else:
#         continue

df_dict['BOREHOLE INTERVAL COLLECTION']['COLLECTION SOURCE DOCUMENT ID'] = np.nan #df_dict['BOREHOLE INTERVAL COLLECTION']['COLLECTION SOURCE DOCUMENT ID'].str.replace('T:','\\\\prod.lan\\active')
df_dict['DOWNHOLE INTERVALS AND SAMPLES']['SOURCE'] = df_dict['DOWNHOLE INTERVALS AND SAMPLES']['SOURCE'].str.replace('T:','\\\\prod.lan\\active')

Loading header details...Compulsory field missing: populate ENO for hole RN029653 before continuing
Compulsory field missing: populate Depth Reference Point Id for hole RN029653 before continuing
Done
Starting Oracle bulk loader processing for \\prod.lan\active\proj\futurex\StuartCorridor\Working\Reuben\Boreholes_stratlith\\Induction + Gamma logs with stratigraphy\East Kimberley\RN029653_SSC.xlsx
Loading log sheet...Done
Creating borehole header...Done
Creating downhole intervals and samples, earth materials and extra properties...Done
RN029653 is processed and ready for loading into Oracle
-----------------------------------
Loading header details...Compulsory field missing: populate ENO for hole RN029656 before continuing
Compulsory field missing: populate Depth Reference Point Id for hole RN029656 before continuing
Done
Starting Oracle bulk loader processing for \\prod.lan\active\proj\futurex\StuartCorridor\Working\Reuben\Boreholes_stratlith\\Induction + Gamma logs with stratigraphy

In [23]:
processed.loc[577,'file_path'] #.tail()
# processed['error'].unique()

'\\\\prod.lan\\active\\proj\\futurex\\StuartCorridor\\Working\\Reuben\\Boreholes_stratlith\\\\Completed\\RN015136_SSC.xlsx'

In [30]:
df = prepareDigitalLog(processed.loc[577,'file_path'])
df


,position: From (m bgl),position: To (m bgl),"lithology: eg. Sand, fine; Clay; interbedded sand and silt and clay etc.",Colour: dominant,Colour: minor 1,Colour: minor 2,Colour: pattern,Colour: Does the colour grade from Dominant to Minor (Y/N),grainsize %: clay,grainsize %: silt,...,assemblage: Unnamed: 48,assemblage: Unnamed: 49,comments: Any additional observations,Extra Fields for Oracle: Sample Type (ie core or cuttings). See lookup tab,"Extra Fields for Oracle: EM1 Lithology Name (eg, soil, muddy sand, sandstone), see lookup tab",Extra Fields for Oracle: EM1 Proportion (%),"Extra Fields for Oracle: EM2 Lithology Name (eg, soil, muddy sand, sandstone)",Extra Fields for Oracle: EM2 Proportion (%),"LogType: Lith Log Type (""driller"", ""geologist"", ""unknown"")",Strat: Stratigraphy
0,0.0,5.0,Sandy Clay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,drill chips/cuttings,sandy clay,NaN,NaN,NaN,Driller,NaN
1,5.0,21.0,Limestone and Clay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,drill chips/cuttings,limestone,NaN,clay,NaN,Driller,NaN
2,21.0,90.0,Granite,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Water Struck at 42.63 m 2 L/sec,drill chips/cuttings,granite,NaN,NaN,NaN,Driller,NaN


In [50]:
df_dict['BOREHOLE INTERVAL COLLECTION'].head()

,BOREHOLE ENO,BOREHOLE NAME,DEPTH REFERENCE POINT ID,INTERVAL COLLECTION ID,COLLECTION NAME,COLLECTION TYPE,ORIGINATOR NUMBER,PREFERRED COLLECTION,COLLECTION SOURCE DOCUMENT ID,SOURCE COMMENTS
0,NaN,RN029653,NaN,,GA Lithological Log,lithological,1328,Y,\\prod.lan\active\proj\futurex\StuartCorridor\...,NaN
1,NaN,RN029656,NaN,,GA Lithological Log,lithological,1328,Y,\\prod.lan\active\proj\futurex\StuartCorridor\...,NaN
2,NaN,RN029660,NaN,,GA Lithological Log,lithological,1328,Y,T:\proj\futurex\StuartCorridor\Working\Reuben\...,NaN
3,NaN,RN029662,NaN,,GA Lithological Log,lithological,1328,Y,T:\proj\futurex\StuartCorridor\Working\Reuben\...,NaN
4,NaN,RN029663,NaN,,GA Lithological Log,lithological,1328,Y,T:\proj\futurex\StuartCorridor\Working\Reuben\...,NaN


In [53]:
df_dict['DOWNHOLE INTERVALS AND SAMPLES']#[df_dict['DOWNHOLE INTERVALS AND SAMPLES'].columns[20:]].head()

,BOREHOLE ENO,COLLECTION NO.,COLLECTION NAME,INTERVAL ID,INTERVALNO,INTERVAL START DEPTH,INTERVAL END DEPTH,SAMPLENO,SAMPLE ID,ACQUISITION DATE,...,MATERIAL CLASS,PROCEDURE NO.,PROJECT,SPECIMEN STORAGE LOCATION,STORAGE DATE,COMMENTS,IGSN,SPECIMEN MASS,MASS UOM,SOURCE
0,NaN,,GA Lithological Log,,,0.0,1.30,,RN029653.lithostrat.1,3000-01-01,...,regolith,NaN,EFTF - South Stuart Corridor,NaN,NaN,Black soil,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
1,NaN,,GA Lithological Log,,,1.3,3.50,,RN029653.lithostrat.2,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Red clay,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
2,NaN,,GA Lithological Log,,,3.5,6.90,,RN029653.lithostrat.3,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Brown silt,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
3,NaN,,GA Lithological Log,,,6.9,7.50,,RN029653.lithostrat.4,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Brown silt/gravels,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
4,NaN,,GA Lithological Log,,,7.5,18.00,,RN029653.lithostrat.5,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Brown sand,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
5,NaN,,GA Lithological Log,,,18.0,18.80,,RN029653.lithostrat.6,3000-01-01,...,rock,NaN,EFTF - South Stuart Corridor,NaN,NaN,Sandstone,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
6,NaN,,GA Lithological Log,,,0.0,1.70,,RN029656.lithostrat.1,3000-01-01,...,regolith,NaN,EFTF - South Stuart Corridor,NaN,NaN,Black soil,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
7,NaN,,GA Lithological Log,,,1.7,6.90,,RN029656.lithostrat.2,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Red clay,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
8,NaN,,GA Lithological Log,,,6.9,18.00,,RN029656.lithostrat.3,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Red clay/brown silt,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...
9,NaN,,GA Lithological Log,,,18.0,22.00,,RN029656.lithostrat.4,3000-01-01,...,sediment,NaN,EFTF - South Stuart Corridor,NaN,NaN,Brown clay with siltstone and gravel,NaN,NaN,NaN,\\prod.lan\active\proj\futurex\StuartCorridor\...


In [54]:
df_dict['EARTH MATERIALS']

,COLLECTION NAME,SAMPLENO,SAMPLE ID,ACCESS CODE,CONFIDENTIAL UNTIL DATE,QA STATUS,ACTIVITY CODE,Unnamed: 7,LITHOLOGY GROUP,LITHOLOGY NAME,QUALIFIER 1,QUALIFIER 2,QUALIFIER 3,MATERIAL DESCRIPTION,MODE OF OCCURRENCE,PROPORTION TERM,LOWER PROPORTION PERCENT,UPPER PROPORTION PERCENT,CONFIDENCE
0,GA Lithological Log,,RN029653.lithostrat.1,O,NaN,U,A,,regolith,soil,,,,,bed,,NaN,NaN,
1,GA Lithological Log,,RN029653.lithostrat.2,O,NaN,U,A,,sedimentary,clay,,,,,bed,,NaN,NaN,
2,GA Lithological Log,,RN029653.lithostrat.3,O,NaN,U,A,,sedimentary,silt,,,,,bed,,NaN,NaN,
3,GA Lithological Log,,RN029653.lithostrat.4,O,NaN,U,A,,sedimentary,gravel,,,,,bed,,NaN,NaN,
4,GA Lithological Log,,RN029653.lithostrat.4,O,NaN,U,A,,sedimentary,silt,,,,,bed,,NaN,NaN,
5,GA Lithological Log,,RN029653.lithostrat.5,O,NaN,U,A,,sedimentary,sand,,,,,bed,,NaN,NaN,
6,GA Lithological Log,,RN029653.lithostrat.6,O,NaN,U,A,,sedimentary,sandstone,,,,,bed,,NaN,NaN,
7,GA Lithological Log,,RN029656.lithostrat.1,O,NaN,U,A,,regolith,soil,,,,,bed,,NaN,NaN,
8,GA Lithological Log,,RN029656.lithostrat.2,O,NaN,U,A,,sedimentary,clay,,,,,bed,,NaN,NaN,
9,GA Lithological Log,,RN029656.lithostrat.3,O,NaN,U,A,,sedimentary,silt,,,,,bed,,NaN,NaN,


In [55]:
df_dict['STRATIGRAPHY']#[df_dict['STRATIGRAPHY'].columns[15:]].head()

,COLLECTION NAME,SAMPLENO,SAMPLE ID,ACCESS CODE,CONFIDENTIAL UNTIL DATE,QA STATUS,ACTIVITY CODE,Unnamed: 7,FORMAL UNIT NAME,INFORMAL UNIT NAME,...,MAX. AGE NAME,AGE METHOD,TOP CONTACT TYPE,TOP CONTACT CHARACTER,TOP CONTACT NAME,BASE CONTACT TYPE,BASE CONTACT CHARACTER,BASE CONTACT NAME,REMARKS,CONFIDENCE


In [56]:
df_dict['EXTRA PROPERTIES'].head()

,SAMPLEID,SAMPLENO,DATANO,PROPERTY,TEXT VALUE,NUMERICAL VALUE,UNIT OF MEASURE,PROPORTION TERM,CONFIDENCE,COMMENT,ACCESS CODE,CONFIDENTIAL UNTIL,QA STATUS,ACTIVITY CODE,ANO
0,RN030826.lithostrat.4,,,colour pattern,mottled,,,,,,O,NaN,U,A,218


In [ ]:
template_path

# save out to new workbook
s = 'Saving output to ' + output_path + '...'
print(s, end = '', flush = True)
writer = pd.ExcelWriter(output_path)
for df_name, df in df_dict.items():
    df.to_excel(writer, df_name, index = False)
writer.save()
print('Done')